# Plots for Needle in Haystack Tests

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import logging
import pickle
from datetime import datetime

#### Experiment Details

In [9]:
valA = "gaussA"
valC = "_gauss_C"
state_dim = 10
ckpt_step = 81000
experiment = "250114_202420.3c1184_multi_sys_trace_gaussA_state_dim_10_gauss_C_lr_1.584893192461114e-05_num_train_sys_40000" # experiment to load
datasource = "train_systems"

errs_dir = "../outputs/GPT2/" + experiment + f"/prediction_errors{valC}_step={ckpt_step}.ckpt"
errs_loc = errs_dir + f"/" + f"needle_{datasource}_" + f"{valA}_state_dim_{state_dim}_"

print(f"Loading errors from {errs_loc}")

Loading errors from ../outputs/GPT2/250114_202420.3c1184_multi_sys_trace_gaussA_state_dim_10_gauss_C_lr_1.584893192461114e-05_num_train_sys_40000/prediction_errors_spec_C_step=81000.ckpt/needle_train_systems_specA_state_dim_10_


#### load the err_lss_examples at path which is numpy array

In [10]:
with open(errs_loc + "err_lss_examples.pkl", "rb") as f:
    err_lss_examples = pickle.load(f)

for key in err_lss_examples.keys():
    print(f"shape of {key}: {err_lss_examples[key].shape}")

FileNotFoundError: [Errno 2] No such file or directory: '../outputs/GPT2/250114_202420.3c1184_multi_sys_trace_gaussA_state_dim_10_gauss_C_lr_1.584893192461114e-05_num_train_sys_40000/prediction_errors_spec_C_step=81000.ckpt/needle_train_systems_specA_state_dim_10_err_lss_examples.pkl'

#### Compute the Quartiles

In [6]:
def comp_quartiles(err_lss_examples, ratio=False):
    quartiles = {}
    kal_err = None
    if ratio:
        kal_err = err_lss_examples["Kalman_rem"]
    

    for key in err_lss_examples.keys():
        if not (key == "Analytical_Kalman" or key == "Kalman_rem" or key == "Kalman"):
            if ratio:
                rat = err_lss_examples[key] / kal_err
            else:
                rat = err_lss_examples[key]
            med = np.median(rat, axis=2)
            print(f"shape of med: {med.shape}")
            quartiles[key] = np.percentile(med, [25,50,75], axis=0)
            print(f"shape of quartiles[{key}]: {quartiles[key].shape}")
            
    return quartiles


In [7]:
quartiles = comp_quartiles(err_lss_examples, ratio=True)

shape of med: (1, 19, 251)
shape of quartiles[MOP]: (3, 19, 251)
shape of med: (1, 19, 251)
shape of quartiles[Zero]: (3, 19, 251)
shape of med: (1, 19, 251)
shape of quartiles[Analytical_Simulation]: (3, 19, 251)
shape of med: (1, 19, 251)
shape of quartiles[OLS_ir_1]: (3, 19, 251)
shape of med: (1, 19, 251)
shape of quartiles[OLS_analytical_ir_1]: (3, 19, 251)
shape of med: (1, 19, 251)
shape of quartiles[OLS_ir_2]: (3, 19, 251)
shape of med: (1, 19, 251)
shape of quartiles[OLS_analytical_ir_2]: (3, 19, 251)
shape of med: (1, 19, 251)
shape of quartiles[OLS_ir_3]: (3, 19, 251)
shape of med: (1, 19, 251)
shape of quartiles[OLS_analytical_ir_3]: (3, 19, 251)


/tmp/ipykernel_343465/341765924.py:11: RuntimeWarning: invalid value encountered in divide
  rat = err_lss_examples[key] / kal_err


## Plotting Code

### final trace

In [ ]:
haystack_len = quartiles["MOP"].shape[1]
#make a figure with haystack_len subplots
fig, ax = plt.subplots(haystack_len, 1, sharex=True, figsize=(40, 250))
print(f"haystack_len: {haystack_len}")
for needle in range(haystack_len):
    for key in quartiles.keys():
        if "OLS_analytical" not in key:
            # ax[needle].scatter(quartiles[key][1, needle], label=key)
            ax[needle].errorbar(np.arange(quartiles[key].shape[-1]), quartiles[key][1, needle], yerr=[quartiles[key][1, needle] - quartiles[key][0, needle], quartiles[key][2, needle] - quartiles[key][1, needle],], fmt='o', label=f"{key}", capsize=2, zorder=haystack_len if key == "MOP" else 0)
    ax[needle].legend()
    ax[needle].grid(which="both")
    ax[needle].set_xlim(left=0, right=quartiles[key].shape[-1])  # set the x axis limits haystack_len*12 + 2
    # ax[needle].set_ylim(bottom=0, top=5.5)  # set the y axis limits

    # Optionally, customize major and minor ticks
    ax[needle].minorticks_on()

    # Set minor vertical grid lines to be on intervals of 1
    # Set major ticks on every interval of 50
    ax[needle].set_xticks(range(int(ax[needle].get_xlim()[0]), int(ax[needle].get_xlim()[1]) + 1, 5))

    # Set minor vertical grid lines to be on intervals of 1
    ax[needle].set_xticks(range(int(ax[needle].get_xlim()[0]), int(ax[needle].get_xlim()[1]) + 1, 1), minor=True)

    ax[needle].tick_params(axis='both', which='major', length=7, width=1, labelsize=20)
    ax[needle].tick_params(axis='both', which='minor', length=4, width=0.5, labelsize=0)
    ax[needle].grid(which='major', linestyle='-', linewidth=1)
    ax[needle].grid(which='minor', linestyle='--', linewidth=0.5)
    ax[needle].set_ylabel("Median Prediction Error", fontsize=20)
    ax[needle].set_xlabel("Context", fontsize=20)
    ax[needle].set_title(f"Prediction Error for Needle Position {needle}", fontsize=30)

#add the date and time to the filename
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")

#add a caption to the bottom of the figure
fig.text(0.5, 0.1, "step=" + str(ckpt_step) + "_" + timestamp, ha='center', fontsize=30)

haystack_len: 19


Text(0.5, 0.1, 'step=81000_20250119_040931')